# Payload

In [2]:
PAYLOAD = {
    "datasetName": "random_10",
    "gaParameters": {
        "maxNumberOfIter":300,
        "populationSize": 100,
        "maximumWildCardCount": 0,
        "orientationWeights": [
            1,
            1,
            0.5
        ],
        "geneticAlgorithm": "simpleGa",
        "mate": "normalizedProbabilityVectorSum",
        "mutate": "flipOnePartAtRandom",
        "select": "tournament",
        "objective": "simple",
        "evaluator": "ga",
        "placingHeuristics": "corner",
        "populationDivisionCounts": {
            "elite": 0.2,
            "average": 0.6,
            "worst": 0.2,
            "children": 0.3,
            "mutant": 0.2,
            "winner": 0.2,
            "random": 0.1
        },
        "initialPopulationDivisionCounts": {
            "random": 0.7,
            "greedy": 0.3
        }
    },
    "objectiveParameters": {
        "name": "simple",
        "params": {
            "overlappingPenalizationConstant": 0,
            "outsideOfAllocatedAreaPenalizationConstant": 0
        }
    }
}

URL = "http://localhost:9000/compute/dataset"

HEADERS = {
  'Content-Type': 'application/json'
}

# Utils

In [7]:
import os
import time
import requests
import json
from pathlib import Path
import os
import shutil
from datetime import datetime
import math
import copy

def createDirectory(directory, removeAll=False):
    '''
    Creates a directory if it doesn't exist and removes all files
    from the directory if it already exists and removeAll is True.
    '''
    if os.path.exists(directory) and removeAll:
        shutil.rmtree(directory) # Remove all files in the directory
        print(f'All files in {directory} have been removed.')

    if not os.path.exists(directory):
        os.makedirs(directory) # Create the directory
        
    return directory

def checkforFileBlocking(outDir):
    start_time = time.time()  # Get the current time in seconds
    while True:
        if os.path.exists(os.path.join(outDir, "computation_result.json")):
            elapsed_time = time.time() - start_time  # Calculate the elapsed time in seconds
            print(f"{elapsed_time:.2f} sec")
            time.sleep(5)  # weit for file to be written and closed
            break
        else:
            time.sleep(1)  # Pause for 1 second before checking again
            
def findDataset(name):
    for root, dirs, files in os.walk('/home/slarty/Documents/master-thesis/src/impl/public/datasets'):
        for file in files:
            if file == f'{name}.json' or file == name:
                return os.path.join(root, file)
    return None

def setOverlappingConstant(data, dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
def sendPayloadBlocking(payload, outDir):
    r = requests.request("POST", URL, headers=HEADERS, data=json.dumps(payload))
    print(r.json())
    src = r.json()['outputDirectory'] 
    checkforFileBlocking(src)
    shutil.copytree(src, outDir + '/' + src.split('/')[-1])
    
def createOutDir(name):
    directory = f'out/{name}_' +  datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
    createDirectory(directory)
    return directory

def printRun(run, runMax):
    print(f'====== RUN {run}/{runMax} ======')

# Measurements

## Population size x best solution

In [ ]:
outDirBase = createOutDir('population_size')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('cluster_3_10', 30), ('cluster_4_20',80)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

## Overlapping Penalization Constant

In [11]:
outDirBase = createOutDir('overlapping_penalization_constant')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

def getLayoutWidthAndHeight(dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    return layout['width'], layout['height']

def calculateDiagonalLength(dataset):
    w,h = getLayoutWidthAndHeight(dataset)
    return math.sqrt(w**2 + h**2)


def setOverlappingConstant(data, dataset):
    with open(findDataset(dataset), 'r') as file:
        layout = json.load(file)['data']['layout']
    data['objectiveParameters']['params']['overlappingPenalizationConstant'] = math.sqrt(layout['width']**2 + layout['height']**2) * 4
    
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_10', 10), ('packing_10', 10)]:
        for k in [2, 3, 4, 6]:
            print(f'{dataset}, {k}')
            payload['datasetName'] = dataset
            payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * k
            payload['gaParameters']['populationSize'] = size * 50
            payload['gaParameters']['maxNumberOfIter'] = 300
            payload['gaParameters']['maximumWildCardCount'] = 0
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
random_10, 2
{'id': 'random_10_D9842', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/094_random_10_D9842'}
113.12 sec
random_10, 3
{'id': 'random_10_73195', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/095_random_10_73195'}
114.11 sec
random_10, 4
{'id': 'random_10_AD7EE', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/096_random_10_AD7EE'}
113.11 sec
random_10, 6
{'id': 'random_10_68D8F', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/097_random_10_68D8F'}
113.11 sec
packing_10, 2
{'id': 'packing_10_1457A', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/098_packing_10_1457A'}
94.09 sec
packing_10, 3
{'id': 'packing_10_B0ACF', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/099_packing_10_B0

## Random

In [20]:
outDirBase = createOutDir('random')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('random_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
{'id': 'random_20_FDC76', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/148_random_20_FDC76'}
990.00 sec

Computation done


## Cluster

In [22]:
outDirBase = createOutDir('cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

# 10.4 - testing maximum wild card count = 1
for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    # for dataset, size in [('cluster_3_6', 18)]:
    for dataset, size in [('cluster_4_5', 20)]:
        print(f'{dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
cluster_4_5
{'id': 'cluster_4_5_606A8', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/150_cluster_4_5_606A8'}
855.86 sec

Computation done


## Packing

In [ ]:
outDirBase = createOutDir('packing')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10), ('packing_20',20)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 1000
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

## Biased sparse cluster

In [25]:
outDirBase = createOutDir('biased_sparse_cluster')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 1

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('biased_sparse_cluster_3_5', 15)]:
        print(f'maximumWildCardCount={maximumWildCardCount}, dataset={dataset}')
        payload['datasetName'] = dataset
        payload['gaParameters']['maxNumberOfIter'] = 300
        payload['gaParameters']['populationSize'] = size * 50
        payload['gaParameters']['maximumWildCardCount'] = 1
        payload['objectiveParameters']['params']['overlappingPenalizationConstant'] = calculateDiagonalLength(dataset) * 8
        sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/1 ======
maximumWildCardCount=2, dataset=biased_sparse_cluster_3_5
{'id': 'biased_sparse_cluster_3_5_F769D', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/152_biased_sparse_cluster_3_5_F769D'}
686.71 sec

Computation done


## Maximum wild card count vs computation time

In [13]:
outDirBase = createOutDir('wild_card_comp_time')
payload = copy.deepcopy(PAYLOAD) 
maxRange = 5

for i in range(maxRange):
    printRun(i+1, maxRange)
    outDir = createDirectory(f'{outDirBase}/{i+1}')
    for dataset, size in [('packing_10',10)]:
        setOverlappingConstant(payload, dataset)
        payload['datasetName'] = dataset
        payload['gaParameters']['populationSize'] = size * 100
        payload['gaParameters']['maxNumberOfIter'] = 300
        
        for maximumWildCardCount in range(0,size+1):
            print(maximumWildCardCount)
            payload['gaParameters']['maximumWildCardCount'] = maximumWildCardCount
            sendPayloadBlocking(payload, outDir)

print('\nComputation done')

====== RUN 1/5 ======
0
{'id': 'packing_10_9D560', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/139_packing_10_9D560'}
180.18 sec
1
{'id': 'packing_10_4B0EB', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/140_packing_10_4B0EB'}
309.31 sec
2
{'id': 'packing_10_94F18', 'outputDirectory': '/home/slarty/Documents/master-thesis/src/impl/target/universal/stage/out/141_packing_10_94F18'}


KeyboardInterrupt: 

## Bruteforce

In [12]:
outDirBase = createOutDir('brute')
payload = copy.deepcopy(PAYLOAD)

data = [('random_10',10), ('random_20',20), ('cluster_3_10',30)]
data = [('cluster_3_6', 18)]

for dataset, size in data:
    print(f'{dataset}')
    setOverlappingConstant(payload, dataset)
    payload['datasetName'] = dataset
    payload['gaParameters']['populationSize'] = 100 * size
    payload['gaParameters']['maxNumberOfIter'] = 1000
    payload['gaParameters']['geneticAlgorithm'] = 'probBrute'
    sendPayloadBlocking(payload, outDirBase)

print('\nComputation done')

random_10
775.77 sec
random_20
3908.93 sec
cluster_3_10
9184.24 sec

Computation done
